In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

In [32]:
final_df = pd.read_csv("../data/final_features_with_targets.csv", parse_dates=['Date'], index_col='Date')

feature_cols = [col for col in final_df.columns if not col.endswith('_target_5d')]
target_cols = [col for col in final_df.columns if col.endswith('_target_5d')]

X = final_df[feature_cols]
y = final_df[target_cols]

In [33]:

def build_model(hp):
    model = keras.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(X_asset.shape[1],)))

    # Tune number of hidden layers: 1 to 3
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"])
        ))
    
    # Output layer
    model.add(layers.Dense(1, activation='linear'))
    
    # Tune optimizer and learning rate
    optimizer = hp.Choice("optimizer", ["adam", "rmsprop", "sgd"])
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    
    return model


In [34]:
tuner = kt.RandomSearch(
    build_model,                 # function previously created 
    objective='val_loss',        # Minimize validation loss
    max_trials=10,               #  different combinations
    executions_per_trial=2,      # Train each model twice and average the results
    directory='kt_mlp_tuning',   # Folder to save search results
    project_name='EEM_MLP'       # Only for one asset EEM, since it's easier to experiment on this 
)

tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

Reloading Tuner from kt_mlp_tuning/EEM_MLP/tuner0.json


In [35]:
best_hp = tuner.get_best_hyperparameters(1)[0]

In [36]:
#We build the model with the best hyper parameter we found in ther previously 
model = build_model(best_hp)


history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
verbose=1
)

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 3.3004 - mae: 1.3745 - val_loss: 0.0209 - val_mae: 0.1310
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0044 - mae: 0.0500 - val_loss: 0.0010 - val_mae: 0.0226
Epoch 3/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4.8782e-04 - mae: 0.0156 - val_loss: 4.1675e-04 - val_mae: 0.0142
Epoch 4/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 3.3443e-04 - mae: 0.0135 - val_loss: 4.4374e-04 - val_mae: 0.0145
Epoch 5/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.9248e-04 - mae: 0.0125 - val_loss: 3.0892e-04 - val_mae: 0.0120
Epoch 6/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.4328e-04 - mae: 0.0117 - val_loss: 3.2910e-04 - val_mae: 0.0131
Epoch 7/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.9430e-04 - mae: 0.0132 - val_loss: 2.9061e-04 - val_mae: 0.0113
Epoch 8/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 2.4574e-04 - mae: 0.0115 - val_loss: 3.4601e-04 - val_mae: 0.0137
Epoch 9/

In [ ]:
# Model prediction 
y_pred = model.predict(X_val)
#Evaluation 

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"RMSE: {rmse:.5f}")
print(f"MAE : {mae:.5f}")
print(f"R²  : {r2:.5f}")

#### Some insights and Comments 
We tuned MLP on just one asset EEM the following were the results
Evaluation Results
RMSE = 0.04231 | MAE = 0.04046 | R² = -10.93108

Even after hyperparameter tuning using Keras Tuner, the MLP model performs poorly on EEM with a highly negative R² score.
This suggests the model fails to capture meaningful return dynamics or overfits during training.
We'll consider regularization or early stopping later, but for now, we move to other models.We will do one batch run for all the assets and see if this pattern follows and then we move to the LSTM and transformer models

